## Basic Bayesian Optimization
In this tutorial we demonstrate the use of Xopt to preform Bayesian Optimization on a
 simple test problem.

## Define the test problem
Here we define a simple optimization problem, where we attempt to minimize the sin
function in the domian [0,2*pi]. Note that the function used to evaluate the
objective function takes a dictionary as input and returns a dictionary as the output.

In [8]:
from xopt.vocs import VOCS
import math

# define variables and function objectives
vocs = VOCS(
    variables={"x": [0.0, 2.0]},
    objectives={"f":"MINIMIZE"},
    observables=["sx","sy"]
)

In [11]:
# define a test function to optimize
import numpy as np

def sin_function(input_dict):
    return {"sx": input_dict["x"]**2, "sy":(input_dict["x"]-2.0)**2,"f":1.0}

## Create Xopt objects
Create the evaluator to evaluate our test function and create a generator that uses
the Upper Confidence Bound acquisition function to perform Bayesian Optimization.

## Generate and evaluate initial points
To begin optimization, we must generate some random initial data points. The first call
to `X.step()` will generate and evaluate a number of randomly points specified by the
 generator. Note that if we add data to xopt before calling `X.step()` by assigning
 the data to `X.data`, calls to `X.step()` will ignore the random generation and
 proceed to generating points via Bayesian optimization.

## Do bayesian optimization steps
To perform optimization we simply call `X.step()` in a loop. This allows us to do
intermediate tasks in between optimization steps, such as examining the model and
acquisition function at each step (as we demonstrate here).

In [12]:
import torch
import matplotlib.pyplot as plt

from xopt.evaluator import Evaluator
from xopt.generators.bayesian import UpperConfidenceBoundGenerator
from xopt import Xopt
from xopt.generators.bayesian.objectives import CustomXoptObjective
from torch import Tensor
from typing import Optional

class MyObjective(CustomXoptObjective):
    def forward(self, samples: Tensor, X: Optional[Tensor] = None) -> Tensor:
        return -torch.sqrt(samples[
            ..., self.vocs.output_names.index("sx")
        ]**2 + samples[
            ..., self.vocs.output_names.index("sy")
        ]**2)
    
class MyObjective2(CustomXoptObjective):
    def forward(self, samples: Tensor, X: Optional[Tensor] = None) -> Tensor:
        return torch.max(samples[
            ..., self.vocs.output_names.index("sx")
        ], samples[
            ..., self.vocs.output_names.index("sy")
        ])
    
evaluator = Evaluator(function=sin_function)
generator = UpperConfidenceBoundGenerator(vocs=vocs, custom_objective=MyObjective2(vocs))
X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)

X.random_evaluate(10)

n_steps = 5

# test points for plotting
test_x = torch.linspace(*X.vocs.bounds.flatten(), 50).double()

for i in range(n_steps):
    # get the Gaussian process model from the generator
    model = X.generator.train_model()

    # get acquisition function from generator
    acq = X.generator.get_acquisition(model)

    # calculate model posterior and acquisition function at each test point
    # NOTE: need to add a dimension to the input tensor for evaluating the
    # posterior and another for the acquisition function, see
    # https://botorch.org/docs/batching for details
    # NOTE: we use the `torch.no_grad()` environment to speed up computation by
    # skipping calculations for backpropagation
    with torch.no_grad():
        posterior = model.posterior(test_x.unsqueeze(1))
        acq_val = acq(test_x.reshape(-1,1, 1))

    # get mean function and confidence regions
    mean = posterior.mean
    l,u = posterior.mvn.confidence_region()

    # plot model and acquisition function
    fig,ax = plt.subplots(2, 1, sharex="all")

    # plot model posterior
    ax[0].plot(test_x, mean, label="Posterior mean")
    #ax[0].fill_between(test_x, l, u, alpha=0.25, label="Posterior confidence region")

    # add data to model plot
    ax[0].plot(X.data["x"],X.data[X.vocs.output_names],"C1o", label="Training data")

    # plot true function

    # add legend
    ax[0].legend()

    # plot acquisition function
    ax[1].plot(test_x, acq_val.flatten())

    ax[0].set_ylabel("f")
    ax[1].set_ylabel(r"$\alpha(x)$")
    ax[1].set_xlabel("x")

    # do the optimization step
    X.step()


C:\Users\rroussel\AppData\Local\mambaforge\envs\xopt-dev\lib\site-packages\botorch\models\utils\assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([0.], dtype=torch.float64), std = tensor([0.], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


RuntimeError: cannot specify objectives in VOCS and a custom objective for the generator

In [4]:
# access the collected data
X.data

,x,sx,sy,xopt_runtime,xopt_error
0,1.496820,2.240471,0.253190,4.500000e-06,False
1,0.072993,0.005328,3.713355,1.400000e-06,False
2,1.198714,1.436916,0.642059,8.000000e-07,False
3,0.956350,0.914605,1.089206,8.000000e-07,False
4,0.768281,0.590256,1.517132,7.000000e-07,False
5,1.850906,3.425852,0.022229,9.000000e-07,False
6,0.029408,0.000865,3.883231,7.000000e-07,False
7,0.467056,0.218141,2.349917,8.000000e-07,False
8,0.143156,0.020494,3.447868,7.000000e-07,False
9,0.732890,0.537128,1.605567,8.000000e-07,False


## Getting the optimization result
To get the best point (without evaluating it) we ask the generator to
predict the optimum based on the posterior mean.

In [5]:
X.generator.get_optimum()

,x
0,0.0


## Customizing optimization
Each generator has a set of options that can be modified to effect optimization behavior

In [6]:
X.generator.dict()

C:\Users\rroussel\AppData\Local\Temp\1\ipykernel_23624\2160990163.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  X.generator.dict()


{'model': ModelListGP(
   (models): ModuleList(
     (0-1): 2 x SingleTaskGP(
       (likelihood): GaussianLikelihood(
         (noise_covar): HomoskedasticNoise(
           (noise_prior): GammaPrior()
           (raw_noise_constraint): GreaterThan(1.000E-04)
         )
       )
       (mean_module): ConstantMean()
       (covar_module): ScaleKernel(
         (base_kernel): MaternKernel(
           (lengthscale_prior): GammaPrior()
           (raw_lengthscale_constraint): Positive()
         )
         (outputscale_prior): GammaPrior()
         (raw_outputscale_constraint): Positive()
       )
       (outcome_transform): Standardize()
       (input_transform): Normalize()
     )
   )
   (likelihood): LikelihoodList(
     (likelihoods): ModuleList(
       (0-1): 2 x GaussianLikelihood(
         (noise_covar): HomoskedasticNoise(
           (noise_prior): GammaPrior()
           (raw_noise_constraint): GreaterThan(1.000E-04)
         )
       )
     )
   )
 ),
 'n_monte_carlo_samples': 1